In [1]:
from pyspark import SparkConf, SparkContext, HiveContext

sc.stop()
conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "8g")
sc = SparkContext(conf=conf)

hc = HiveContext(sc)

table_name = 'user_kposminin.urls_w_levels1'
query2 = '''
select * from {0}

'''.format(table_name)

#hc.sql(query2[0])
#hc.sql(query2[1])

rdd = hc.sql(query2).rdd #Посмотреть


In [2]:
#Group by cookie and form url list the cookie visited
rdd2 = rdd.map(lambda row: (row['cookie'] , row['domain'])).reduceByKey(lambda a,b: a + ';;' + b)

# Label target cookies (=cookies visited target url) and exclude some urls
import re
#target_urls =['mkb.ru/facility/private_person/cards/credit_card','mkb.ru']
target_urls =['avito.ru']
exclude_urls = target_urls + []

def handle_row(row,targ_urls, exclud_urls):
    proc_urls = row[1]
    for u in exclud_urls:
        proc_urls = re.sub('[^;;]*'+ u +'[^;;]*','',proc_urls)
    return (
        any(tu in row[1] for tu in targ_urls), 
        re.sub('[;]{3,}',';;',proc_urls)
    )

rdd3 = rdd2.map(lambda row: handle_row(row,target_urls, exclude_urls))

from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel

tf = HashingTF(numFeatures = 10 ** 4)

#transform urls (as Bag of Words) into features and form features with labels
all_data = rdd3.map(lambda row: LabeledPoint(row[0], tf.transform(row[1].split(';;'))))
# TODO count visits or not
# split into train and test samples
train_data, test_data = all_data.randomSplit([6, 4])

In [13]:
#Train model

# Don't work like this. Naive Bayes
train_data.cache()
methods = [
    {'name': 'LogisticRegression', 'method': LogisticRegressionWithSGD},
    {'name': 'NaiveBayes', 'method': NaiveBayes}
]


for m in methods:
    m['model'] = m['method'].train(train_data)
    m['model'].clearThreshold()

test_res = test_data.map(lambda lp: [lp.label] + [m['model'].predict(lp.features) for m in methods]).toDF().toPandas()
test_res.columns = ['Label'] + [m['name'] for m in methods]

import sklearn
for m in methods:
    m['aucroc'] = sklearn.metrics.roc_auc_score(test_res['Label'],test_res[m['name']])



AttributeError: 'NaiveBayesModel' object has no attribute 'clearThreshold'

In [16]:
methods[1]['model'].predict([e.features for e in test_data.top(20)])

ValueError: Index 10000 out of bounds.

In [9]:
print('Methods quality:\n'.join(['{0} AUC ROC is {1:.5%}'.format(m['name'],m['aucroc']) for m in methods]))

KeyError: 'aucroc'

In [ ]:
def calc_ROC(arr):
    ''''Calculates ROC curve from arr input with two columns: 
        first is classifier score and second is label (0 or 1). arr is sorted by score.
        Output is tuple with AUC ROC as first element and ROC array with two 
        columns FPR and TPR as second element.
    '''
    #cumulative positive, negative, true positive rate, false positive rate,
    # area under ROC curve
    TPR,FPR, AUC = 0., 0., 0.
    one_to_P = 1. / sum([e[1] for e in arr])
    one_to_N =  one_to_P / (len(arr) *  one_to_P - 1)
    ROC = []
    for e in arr:
        if e[1] == 0 :            
            FPR+= one_to_N
            AUC += TPR * one_to_N
        else:            
            TPR+= one_to_P
        ROC.append([FPR,TPR])
    return (AUC, ROC)
    
#test_res_list = sorted(test_res_list, key = lambda e: -e[0])
   

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#plt.scatter([e[0] for e in res[1]],[e[0] for e in res[1]])